Data selection from an Excel file with xlrd and pandas
----------------------------------------------------------

I wanted to explore the ITRF positions to see if I can be interested in some positions. Unfortunately, the [site](http://www.enseignementsup-recherche.gouv.fr/cid23693/s-inscrire-aux-concours-ingenieurs-et-personnels-techniques-de-recherche-et-de-formation-externes-et-internes-de-categorie-a.html#Liste_des_emplois_I.T.R.F._de_categorie_A) is quite awful and the list of positions consist of a excel file with more than 1000 rows.

Of course I am not interested in this ~1000 positions, so I wrote this script to extract the positions I am interested in.

Pandas cannot read hyperlinks, so I used xlrd

In [1]:
%matplotlib inline
import xlrd
import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib
#matplotlib.style.use('ggplot')
from IPython.display import IFrame # To visualize PDFs

pd.set_option('max_colwidth',200)  # put a long limit otherwise pandas will cut the long URLs
pd.options.display.max_rows = 8

In [2]:
book = xlrd.open_workbook("EMPLOIS_ITRF_CAT_A_2016_v1_556960.xls")

In [3]:
sheet = book.sheet_by_index(0)

In [4]:
sheet.cell_value(rowx=100,colx=7)

'-> WebITRF'

In [5]:
col_name = sheet.row_values(10)
column_name = []
for name in col_name:
    column_name.append(name.lower())
column_name

['bap',
 'corps',
 'emploi-type',
 'nature',
 'centre organisateur du concours',
 'affectataire\n(établissement ou service recruteur)',
 'localisation',
 'coordonnees organisateur et affectataire & calendrier']

In [6]:
sheet.row_values(100)

['B',
 "INGENIEUR D'ETUDES 2E CLASSE",
 "Ingénieur en techniques d'analyse chimique",
 'Externe',
 'UNIVERSITÉ PARIS-EST CRÉTEIL VAL DE MARNE',
 'UNIVERSITE ST ETIENNE         ',
 'ST ETIENNE (042)',
 '-> WebITRF']

In [7]:
sheet.nrows

1053

In [8]:
sheet.ncols

8

#### I want to extract the hyperlink "-> WebITRF"

In [9]:
df = pd.DataFrame(columns=(column_name))
df.rename(columns={'coordonnees organisateur et affectataire & calendrier' : "link"}, inplace=True)
df.rename(columns={'centre organisateur du concours' : "centre-organisateur"}, inplace=True)
df.rename(columns={'affectataire\n(établissement ou service recruteur)' : "affectation"}, inplace=True)
df

,bap,corps,emploi-type,nature,centre-organisateur,affectation,localisation,link


In [10]:
df.loc[0]= sheet.row_values(100)
df

,bap,corps,emploi-type,nature,centre-organisateur,affectation,localisation,link
0,B,INGENIEUR D'ETUDES 2E CLASSE,Ingénieur en techniques d'analyse chimique,Externe,UNIVERSITÉ PARIS-EST CRÉTEIL VAL DE MARNE,UNIVERSITE ST ETIENNE,ST ETIENNE (042),-> WebITRF


In [11]:
row_start = 11   # 9 lines are empty?
for row in range(row_start, sheet.nrows):
    rowValues = sheet.row_values(row, start_colx=0, end_colx=sheet.ncols)
    bap = rowValues[0]
    corps = rowValues[1]
    emploi_type = rowValues[2]
    nature = rowValues[3]
    centre_organisateur = rowValues[4]
    centre_recruteur = rowValues[5]
    localisation = rowValues[6]
    # column 7:
    link = sheet.hyperlink_map.get((row, sheet.ncols-1))
    url = '(No URL)' if link is None else link.url_or_path
    
    #print(corps.ljust(10) + ': ' + url)
    # Append the row to the data frame:
    df.loc[row-row_start] = [bap, corps, emploi_type, nature, centre_organisateur, centre_recruteur, localisation, url]

In [12]:
df

,bap,corps,emploi-type,nature,centre-organisateur,affectation,localisation,link
0,A,INGENIEUR DE RECHERCHE 2E CLASSE,Ingénieur en biologie,Externe,UNIVERSITÉ DE BORDEAUX,UNIVERSITE CLERMONT FERRAND 1,CLERMONT FERRAND (063),http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=A1A23IR2EB10631262E
1,A,INGENIEUR DE RECHERCHE 2E CLASSE,Ingénieur en biologie,Externe,UNIVERSITÉ DE BORDEAUX,UNIVERSITE CLERMONT FERRAND 1,CLERMONT FERRAND (063),http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=A1A23IR2EB10631262E
2,A,INGENIEUR DE RECHERCHE 2E CLASSE,Ingénieur en biologie,Externe,UNIVERSITÉ DE BORDEAUX,UNIVERSITE D'AMIENS,AMIENS (080),http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=A1A23IR2EB10801344B
3,A,INGENIEUR DE RECHERCHE 2E CLASSE,Ingénieur en biologie,Externe,UNIVERSITÉ DE BORDEAUX,UNIVERSITE DE LORRAINE,VANDOEUVRE LES NANCY (054),http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=A1A23IR2EB10542493S
...,...,...,...,...,...,...,...,...
1038,J,ASSISTANTS INGENIEURS (RECH. ET FORM.),Assistant en gestion administrative,Interne,ECOLE NORMALE SUPERIEURE DE CACHAN,UNIVERSITE PARIS 9,PARIS 16 (075),http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=J3X21ASIISC0750736T
1039,J,ASSISTANTS INGENIEURS (RECH. ET FORM.),Assistant en gestion administrative,Interne,ECOLE NORMALE SUPERIEURE DE CACHAN,UNIVERSITE PARIS 9,PARIS 16 (075),http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=J3X21ASIISC0750736T
1040,J,ASSISTANTS INGENIEURS (RECH. ET FORM.),Assistant en gestion administrative,Interne,ECOLE NORMALE SUPERIEURE DE CACHAN,UNIVERSITE STRASBOURG,STRASBOURG (067),http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=J3X21ASIISC0673021V
1041,J,ASSISTANTS INGENIEURS (RECH. ET FORM.),Assistant en gestion administrative,Interne,ECOLE NORMALE SUPERIEURE DE CACHAN,UNIVERSITE TOULOUSE 2,TOULOUSE (031),http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=J3X21ASIISC0311383K


#### What are the different "corps"?

In [13]:
corps = set([corps for corps in df['corps']]); corps

{'ASSISTANTS INGENIEURS (RECH. ET FORM.)',
 "INGENIEUR D'ETUDES 2E CLASSE",
 'INGENIEUR DE RECHERCHE 1E CLASSE',
 'INGENIEUR DE RECHERCHE 2E CLASSE'}

#### What are the different type of jobs?

In [14]:
job_type = set([job for job in df['emploi-type']]); job_type

{"Administrateur de systèmes d'information",
 'Administrateur systèmes informatiques, réseaux et télécommunications',
 "Architecte des systèmes d'information",
 'Assistant cartographe',
 "Assistant d'orientation et d'insertion professionnelle",
 'Assistant de communication',
 'Assistant de fabrication',
 "Assistant de l'action culturelle",
 'Assistant de médiation scientifique',
 'Assistant de ressources documentaires',
 'Assistant de rédaction/édition',
 "Assistant des métiers de l'image et du son",
 'Assistant en expérimentation et instrumentation biologiques',
 'Assistant en fabrication mécanique',
 'Assistant en gestion administrative',
 'Assistant en gestion des ressources humaines',
 'Assistant en gestion financière et comptable',
 'Assistant en hygiène et sécurité',
 'Assistant en ingénierie de formation continue',
 'Assistant en instrumentation scientifique et techniques expérimentales',
 'Assistant en production et analyse de données',
 "Assistant en technIques d'élevage et pr

### Basic analysis

Compare number of position per type of "corps" and in each "emploi-type" and "Externe" Vs "Interne" 

In [15]:
ir1 = len(df[(df.corps == "INGENIEUR DE RECHERCHE 1E CLASSE")])/len(df)*100
ir2 = len(df[(df.corps == "INGENIEUR DE RECHERCHE 2E CLASSE")])/len(df)*100
print("Number of IR 1 positions: {0:d}".format(len(df[(df.corps == "INGENIEUR DE RECHERCHE 1E CLASSE")])))
print("Pourcentage of IR 1 position: {0:.2f}".format(ir1))
print("Number of IR 2 positions: {0:d}".format(len(df[(df.corps == "INGENIEUR DE RECHERCHE 2E CLASSE")])))
print("Pourcentage of IR 2 position: {0:.2f}".format(ir2))

Number of IR 1 positions: 6
Pourcentage of IR 1 position: 0.58
Number of IR 2 positions: 154
Pourcentage of IR 2 position: 14.78


### Classification based on the "corps" and job type

Combining "corps" and job type:

In [16]:
my_corps_selection = ["INGENIEUR DE RECHERCHE 1E CLASSE", "INGENIEUR DE RECHERCHE 2E CLASSE"]

In [17]:
my_job_type = ["Chef de projet ou expert en calcul scientifique", 
               "Chef de projet ou expert en information statistique",
               "Chef de projet ou expert en développement et déploiement d'applications",
               "Gestionnaire de base de données",
               "Ingénieur de recherche en traitement et analyse de bases de données",
               "Ingénieur en analyse de données biologiques",
               "Ingénieur en calcul scientifique",
               "Ingénieur en développement et déploiement d'applications",
               "Ingénieur en traitement de données biologiques",
               "Ingénieur statisticien"
              ]

In [18]:
#df[((df.corps == "INGENIEUR DE RECHERCHE 1E CLASSE") | (df.corps == "INGENIEUR DE RECHERCHE 2E CLASSE")) ]

In [19]:
my_df = df[(df['corps'].isin(my_corps_selection)) & (df['emploi-type'].isin(my_job_type)) & (df['nature'] == 'Externe')].copy(); my_df

,bap,corps,emploi-type,nature,centre-organisateur,affectation,localisation,link
18,A,INGENIEUR DE RECHERCHE 2E CLASSE,Ingénieur en analyse de données biologiques,Externe,UNIVERSITÉ DE STRASBOURG,MUSEUM NAT.HIST.NATUREL.PARIS,PARIS 16 (075),http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=A1A21IR2ES10753494R
176,D,INGENIEUR DE RECHERCHE 2E CLASSE,Ingénieur de recherche en traitement et analyse de bases de données,Externe,UNIVERSITÉ DE CERGY-PONTOISE,UNIVERSITE PARIS 7,PARIS 13 (075),http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=D1A21IR2ECP0751723R
196,E,INGENIEUR DE RECHERCHE 1E CLASSE,Chef de projet ou expert en développement et déploiement d'applications,Externe,UNIVERSITÉ DE PICARDIE JULES VERNE,UNIVERSITE LYON 2,BRON (069),http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1B22IR1EAM0691775E
205,E,INGENIEUR DE RECHERCHE 2E CLASSE,Chef de projet ou expert en information statistique,Externe,UNIVERSITE DU HAVRE,ADMINISTRATION CENTRALE MEN,PARIS 7 (075),http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1D24IR2ELH0752310D
...,...,...,...,...,...,...,...,...
223,E,INGENIEUR DE RECHERCHE 2E CLASSE,Chef de projet ou expert en développement et déploiement d'applications,Externe,UNIVERSITÉ DE PICARDIE JULES VERNE,RECTORAT DE VERSAILLES,GUYANCOURT (ACAD. VERSAILLES),http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1B22IR2EAM0780503Y
224,E,INGENIEUR DE RECHERCHE 2E CLASSE,Chef de projet ou expert en développement et déploiement d'applications,Externe,UNIVERSITÉ DE PICARDIE JULES VERNE,RECTORAT D'ORLEANS,ORLEANS (045),http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1B22IR2EAM0450080T
225,E,INGENIEUR DE RECHERCHE 2E CLASSE,Chef de projet ou expert en développement et déploiement d'applications,Externe,UNIVERSITÉ DE PICARDIE JULES VERNE,UNIVERSITE GRENOBLE ALPES,ST MARTIN D HERES (038),http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1B22IR2EAM0383493R
226,E,INGENIEUR DE RECHERCHE 2E CLASSE,Chef de projet ou expert en développement et déploiement d'applications,Externe,UNIVERSITÉ DE PICARDIE JULES VERNE,UNIVERSITE PARIS 5,PARIS 6 (075),http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1B22IR2EAM0751721N


In [20]:
print("The number of potential positions that I am interested in is: {}".format(len(my_df)))

The number of potential positions that I am interested in is: 24


In [21]:
#df[196:198]['link']

In [22]:
#my_df[0:8][['localisation','link']]
my_df[0:8]['link']

18     http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=A1A21IR2ES10753494R
176    http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=D1A21IR2ECP0751723R
196    http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1B22IR1EAM0691775E
205    http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1D24IR2ELH0752310D
207    http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1E25IR2ET30631262E
208    http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1E25IR2ET30801344B
209    http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1E25IR2ET30595964M
210    http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1E25IR2ET30691774D
Name: link, dtype: object

In [23]:
#my_df[8:16][['localisation','link']]
my_df[8:16]['link']

211    http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1E25IR2ET30941111X
212    http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1E25IR2ET30751722P
213    http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1E25IR2ET30751722P
214    http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1E25IR2ET30751722P
215    http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1E25IR2ET30751722P
216    http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1E25IR2ET30751723R
217    http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1E25IR2ET30761904G
218    http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1E25IR2ET30761904G
Name: link, dtype: object

In [24]:
#my_df[16:24][['localisation','link']]
my_df[16:24]['link']

219    http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1E25IR2ET30673021V
220    http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1B22IR2EAM0130178Y
221    http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1B22IR2EAM0753291V
222    http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1B22IR2EAM0350063D
223    http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1B22IR2EAM0780503Y
224    http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1B22IR2EAM0450080T
225    http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1B22IR2EAM0383493R
226    http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1B22IR2EAM0751721N
Name: link, dtype: object

After reading the description of the job associated to the previous links, I remove most of the "first choices" to keep only 2 choices:

In [25]:
rows2remove = [176, 196, 205, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226]

In [26]:
my_df = my_df.drop(rows2remove); my_df

,bap,corps,emploi-type,nature,centre-organisateur,affectation,localisation,link
18,A,INGENIEUR DE RECHERCHE 2E CLASSE,Ingénieur en analyse de données biologiques,Externe,UNIVERSITÉ DE STRASBOURG,MUSEUM NAT.HIST.NATUREL.PARIS,PARIS 16 (075),http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=A1A21IR2ES10753494R
207,E,INGENIEUR DE RECHERCHE 2E CLASSE,Chef de projet ou expert en calcul scientifique,Externe,UNIVERSITE PAUL SABATIER TOULOUSE 3,UNIVERSITE CLERMONT FERRAND 1,CLERMONT FERRAND (063),http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1E25IR2ET30631262E


In [27]:
my_df['link']

18     http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=A1A21IR2ES10753494R
207    http://www.itrf.education.gouv.fr/itrf/resultats.do?CodeConcours=E1E25IR2ET30631262E
Name: link, dtype: object

#### Adding a column containing the description

In [30]:
pdf_description = ['Description_poste/18.pdf',
                   'Description_poste/207.pdf'
                  ]

In [31]:
my_df['description'] = pdf_description

In [32]:
my_df['description'].iloc[0]

'Description_poste/18.pdf'

### Job description: show the PDFs

In [33]:
IFrame(my_df['description'].iloc[0], width=980, height=700)

In [36]:
IFrame(my_df['description'].iloc[1], width=980, height=700)

It would be nice to include this kind of selection in a website so all people interested in this kind of position could pre-select what they are looking for. #to_do_when_I_have_time